# Load data, visualize domain, material, bcs and fields

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from ipywidgets import interactive, widgets, interact

import os
import numpy as np
import matplotlib.pyplot as plt

notebook_path = os.path.abspath("visualize_data.ipynb")

filename_base = None
folder = None
problem = None

def SelectFile(File):
    
    global filename_base
    
    global domain
    global surface
    global bcs
    global material
    global field
    
    global mesh
    global kappa
    global steps
    
    filename_base = File
        
    domain_pathname = folder+"/domain_"+filename_base+".npy"
    domain_surf_pathname = folder+"/domain_surf_"+filename_base+".npy"
    bcs_pathname = folder+"/bcs_"+filename_base+".npy"
    bcs_surf_pathname = folder+"/bcs_surf_"+filename_base+".npy"
    material_pathname = folder+"/material_"+filename_base+".npy"

    mesh_pathname = folder+"/mesh_"+filename_base+".npy"
    kappa_pathname = folder+"/kappa_"+filename_base+".npy"
    steps_pathname = folder+"/steps_"+filename_base+".npy"
    field_pathname = folder+"/field_"+filename_base+".npy"
    

    
    domain = np.load(domain_pathname)
    domain_surf = np.load(domain_surf_pathname)
    bcs = np.load(bcs_pathname)
    bcs_surf = np.load(bcs_surf_pathname)
    material = np.load(material_pathname)
    field = np.load(field_pathname)

    mesh = np.load(mesh_pathname)
    kappa = np.load(kappa_pathname)
    steps = np.load(steps_pathname)

    bc_domain = bcs_surf - domain_surf   



    fig1, axs1 = plt.subplots(3, 2, figsize=(30,24))
    fig1.suptitle('Domain and Surface Masks')

    axs1[0,0].set_title('Domain')
    axs1[0,1].set_title('Domain_Surf')
    axs1[1,0].set_title('BCs')
    axs1[1,1].set_title('BCs_Surf')
    axs1[2,0].set_title('Surface')
    axs1[2,1].set_title('Material')

    im1 = []
    im1.append(axs1[0,0].imshow(domain, origin='lower'))
    im1.append(axs1[0,1].imshow(domain_surf, origin='lower'))
    im1.append(axs1[1,0].imshow(bcs, origin='lower'))
    im1.append(axs1[1,1].imshow(bcs_surf, origin='lower'))
    im1.append(axs1[2,0].imshow(bc_domain, origin='lower'))
    im1.append(axs1[2,1].imshow(material, origin='lower'))
    
    fig1.colorbar(im1[0], ax=axs1[0,0])
    fig1.colorbar(im1[1], ax=axs1[0,1])
    fig1.colorbar(im1[2], ax=axs1[1,0])
    fig1.colorbar(im1[3], ax=axs1[1,1])
    fig1.colorbar(im1[4], ax=axs1[2,0])
    fig1.colorbar(im1[5], ax=axs1[2,1])

    cval = steps[-1,:]
    cval = kappa
    cval = steps[0,:]

    x_size = field.shape[2]
    y_size = field.shape[1]
    aspect = x_size/y_size

    print('domain sum: '+str(np.sum(domain[50,:])))
    print('domain_surf sum: '+str(np.sum(domain_surf[50,:])))

    print('domain: '+str(domain.shape))
    print('domain_surf: '+str(domain_surf.shape))
    print('bcs: '+str(bcs.shape))
    print('bcs_surf: '+str(bcs_surf.shape))
    print('material: '+str(material.shape))
    print('min material: '+str(np.min(material)))
    print('max material: '+str(np.max(material)))
    print('field: '+str(field.shape))
    print('steps: '+str(steps.shape))
    print('kappa: '+str(kappa.shape))
    

def SelectProblem(Problem):
    global folder
    global problem
    problem = Problem
    folder = "data/"+Problem
    folderPath = os.path.join(os.path.dirname(notebook_path), "data/"+Problem)
    files = []
    for file in os.listdir(folderPath):
        if file.endswith(".json"):
            files.append(file[7:-5])

    interact(SelectFile, File=files)
        
# interact(SelectProblem, Problem=['datasets_train_waveequation', 'datasets_test_waveequation', 'datasets_train_heatequation', 'datasets_test_heatequation'])
interact(SelectProblem, Problem=['datasets_test_heatequation', 'datasets_train_heatequation', 'datasets_test_waveequation', 'datasets_train_waveequation'])

print('')

# Visualize field over time (2d)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import plotly
from ipywidgets import interactive, HBox, VBox, widgets, interact
py.init_notebook_mode()

if "heatequation" in problem:
    zLimit = 1.0
    zmin = 0
    zmax = zLimit
else:
    zLimit = 0.05
    zmin = -zLimit
    zmax = zLimit

aspect = field.shape[2]/field.shape[1]
imgHeatmap = go.Heatmap(z=field[0,:,:], zmin=zmin, zmax=zmax)
figHeatmap = go.Figure(data=[imgHeatmap])
figHeatmap.update_layout(yaxis = dict(scaleanchor = 'x'))

sliderHeatmap = widgets.IntSlider(
    min=0,
    max=field.shape[0]-1,
    step=1,
    readout=False,
    description='Timestep')
sliderHeatmap.layout.width = '800px'

fHeatmap = go.FigureWidget(data=figHeatmap.data, layout=figHeatmap.layout)

def updateHeatmap(x):
    print(x)
    fHeatmap.data[0].z = field[x,:,:]

vbHeatmap = VBox((fHeatmap, interactive(updateHeatmap, x=sliderHeatmap)))
vbHeatmap.layout.align_items = 'center'
vbHeatmap

# Visualize field over time (3d)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import plotly
from ipywidgets import interactive, HBox, VBox, widgets, interact
py.init_notebook_mode()


if "heatequation" in problem:
    zLimit = 1.0
else:
    zLimit = 0.5
aspect = field.shape[2]/field.shape[1]
surf = go.Surface(z=field[0,:,:], cmin=-zLimit, cmax=zLimit)
figSurf = go.Figure(data=[surf])
figSurf.update_layout(scene_aspectmode='manual',
                  scene_aspectratio=dict(x=aspect, y=1, z=1))
figSurf.update_layout(scene = dict(
                    xaxis = dict(nticks=4, range=[0,field.shape[2]],),
                    yaxis = dict(nticks=4, range=[0,field.shape[1]],),
                    zaxis = dict(nticks=4, range=[-zLimit,zLimit],),),)

sliderSurf = widgets.IntSlider(
    min=0,
    max=field.shape[0]-1,
    step=1,
    readout=False,
    description='Timestep')
sliderSurf.layout.width = '800px'

fSurf = go.FigureWidget(data=figSurf.data, layout=figSurf.layout)

def updateSurf(x):
    fSurf.data[0].z = field[x,:,:]

vbSurf = VBox((fSurf, interactive(updateSurf, x=sliderSurf)))
vbSurf.layout.align_items = 'center'
vbSurf